# Supervised ANNs for Mutual Information Investigation
## Thomas Possidente

### Imports and Initializations

In [1]:
# ANN Building and Visualization Imports
import keras
from keras import backend as K
from keras import optimizers, losses
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from keras.callbacks import LambdaCallback
from keras.utils import to_categorical
import matplotlib.pyplot as plt



# Standard Imports
import pandas as pd
import numpy as np
import math

# Value Inits - Specify as needed
num_inputs = int(5000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(2)        # Dimensions of the RF to be analyzed should be RF_size*RF_size
noise = 0.1              # percentage (as decimal) of input values that will be flipped 

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading in Data

In [30]:
inputs = pd.read_csv('test.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(num_inputs, size, size,1) # reshape to desired dims (5000 examples, 16*16 image, 1 channel)
flattened_inputs = inputs.reshape(num_inputs, size*size)

### Creating Labels

In [32]:
labels = np.zeros(np.shape(flattened_inputs)[0])
storage = np.zeros(np.shape(flattened_inputs))
count = 0

for i in range(np.shape(flattened_inputs)[0]):
    if(~((flattened_inputs[i] == storage).all(1).any())):
        labels[i] = count
        storage[i] = flattened_inputs[i]
        count += 1
    elif((flattened_inputs[i] == storage).all(1).any()):
        label_index = np.where((flattened_inputs[i] == storage).all(1))
        correct_label = labels[label_index]
        storage[i] = flattened_inputs[i]
        labels[i] = correct_label[0]
        
labels = labels.astype(int)
labels = to_categorical(labels)

    

In [5]:
labels.shape

(5120, 256)

### Cutting Number of Labels Possible to 256 if Necessary

In [31]:
inputs = inputs[0:256]
inputs = np.repeat(inputs, 20, axis=0)
inputs = np.random.permutation(inputs)
flattened_inputs = flattened_inputs[0:256]
flattened_inputs = np.repeat(flattened_inputs, 20, 0)
flattened_inputs = np.random.permutation(flattened_inputs)

num_inputs = flattened_inputs.shape[0]


### Adding Noise to Inputs

In [33]:
for n in range(np.shape(flattened_inputs)[0]):
    indices_to_flip = np.random.choice(int(size*size), math.ceil(size*size * noise), replace = False)
    flattened_inputs[n][indices_to_flip] = 1 - flattened_inputs[n][indices_to_flip]

inputs = flattened_inputs.reshape(num_inputs, size, size, 1)


In [2]:
model = Sequential()
model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 4)           20        
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
Total params: 262,932
Trainable params: 262,932
Non-trainable params: 0
_________________________________________________________________


### Building Network

In [35]:
def run_model():
    
    model = Sequential()
    model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(256, activation = 'softmax'))
    model.compile(optimizer = optimizers.adam(lr = 0.0005), loss = 'categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x = inputs, validation_split = 0.25, y = labels, batch_size = 100, epochs = 25, shuffle=False)
    return(history)



In [36]:
val_acc15 = np.zeros(10)
val_acc99 = np.zeros(10)
val_loss15 = np.zeros(10)
val_loss99 = np.zeros(10)
for i in range(10):
        results = run_model()
        val_acc15[i] = results.history['val_acc'][4]
        val_acc99[i] = results.history['val_acc'][24]
        val_loss15[i] = results.history['val_loss'][4]
        val_loss99[i] = results.history['val_loss'][24]

print("acc15 = " + str(np.mean(val_acc15)), "\n",
      "acc100 = " + str(np.mean(val_acc99)), "\n",
      "loss15 = " + str(np.mean(val_loss15)), "\n",
     "loss100 = " + str(np.mean(val_loss99)))

Train on 3840 samples, validate on 1280 samples
Epoch 1/25
3840/3840 [==============================] - 4s 975us/step - loss: 5.5439 - acc: 0.0065 - val_loss: 5.5374 - val_acc: 0.0047
Epoch 2/25
3840/3840 [==============================] - 1s 161us/step - loss: 5.5251 - acc: 0.0128 - val_loss: 5.5121 - val_acc: 0.0141
Epoch 3/25
3840/3840 [==============================] - 1s 163us/step - loss: 5.4844 - acc: 0.0172 - val_loss: 5.4689 - val_acc: 0.0141
Epoch 4/25
3840/3840 [==============================] - 1s 160us/step - loss: 5.4267 - acc: 0.0240 - val_loss: 5.4199 - val_acc: 0.0164
Epoch 5/25
3840/3840 [==============================] - 1s 169us/step - loss: 5.3608 - acc: 0.0312 - val_loss: 5.3630 - val_acc: 0.0234
Epoch 6/25
3840/3840 [==============================] - 1s 168us/step - loss: 5.2851 - acc: 0.0453 - val_loss: 5.2955 - val_acc: 0.0344
Epoch 7/25
3840/3840 [==============================] - 1s 170us/step - loss: 5.1979 - acc: 0.0604 - val_loss: 5.2162 - val_acc: 0.0398


Epoch 10/25
3840/3840 [==============================] - 1s 190us/step - loss: 4.9987 - acc: 0.1445 - val_loss: 5.0331 - val_acc: 0.1008
Epoch 11/25
3840/3840 [==============================] - 1s 185us/step - loss: 4.8819 - acc: 0.1799 - val_loss: 4.9228 - val_acc: 0.1258
Epoch 12/25
3840/3840 [==============================] - 1s 179us/step - loss: 4.7546 - acc: 0.2206 - val_loss: 4.8015 - val_acc: 0.1594
Epoch 13/25
3840/3840 [==============================] - 1s 181us/step - loss: 4.6179 - acc: 0.2656 - val_loss: 4.6708 - val_acc: 0.1977
Epoch 14/25
3840/3840 [==============================] - 1s 184us/step - loss: 4.4735 - acc: 0.3063 - val_loss: 4.5324 - val_acc: 0.2227
Epoch 15/25
3840/3840 [==============================] - 1s 158us/step - loss: 4.3228 - acc: 0.3414 - val_loss: 4.3881 - val_acc: 0.2555
Epoch 16/25
3840/3840 [==============================] - 1s 165us/step - loss: 4.1678 - acc: 0.3760 - val_loss: 4.2400 - val_acc: 0.2820
Epoch 17/25
3840/3840 [==================

3840/3840 [==============================] - 1s 268us/step - loss: 5.4904 - acc: 0.0154 - val_loss: 5.4840 - val_acc: 0.0164
Epoch 4/25
3840/3840 [==============================] - 1s 284us/step - loss: 5.4436 - acc: 0.0232 - val_loss: 5.4465 - val_acc: 0.0219
Epoch 5/25
3840/3840 [==============================] - 1s 288us/step - loss: 5.3908 - acc: 0.0320 - val_loss: 5.4037 - val_acc: 0.0250
Epoch 6/25
3840/3840 [==============================] - 1s 301us/step - loss: 5.3302 - acc: 0.0422 - val_loss: 5.3532 - val_acc: 0.0281
Epoch 7/25
3840/3840 [==============================] - 1s 263us/step - loss: 5.2604 - acc: 0.0599 - val_loss: 5.2934 - val_acc: 0.0328
Epoch 8/25
3840/3840 [==============================] - 1s 265us/step - loss: 5.1806 - acc: 0.0753 - val_loss: 5.2232 - val_acc: 0.0430
Epoch 9/25
3840/3840 [==============================] - 1s 289us/step - loss: 5.0903 - acc: 0.0966 - val_loss: 5.1421 - val_acc: 0.0547
Epoch 10/25
3840/3840 [==============================] - 1s

In [195]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

### TODOS
* Find Image dataset that's binarized (or binarize existing one) and do MI calculations to find max MI. To do this, sample RF sized chunks of an image until stable average MI value is reached
* https://people.cs.umass.edu/~marlin/data.shtml, http://vision.lems.brown.edu/content/available-software-and-databases